In [1]:
%load_ext autoreload

In [20]:
%autoreload

import numpy as np
from astropy.io import fits
from scipy.interpolate import interp1d
import math as m
import scipy.constants as p
import pandas as pd
import matplotlib.pyplot as plt
import timeit
import sys
sys.path.append("..")
import physics
sys.path.append("../..")
from spec.spectrum import *
#%run 'Electrons_v1.ipynb' might need things form this later
import xsecs
np.set_printoptions(precision=20)

In [24]:
class Rates(object):
    '''
    Allows for easier storage of all xsecs/rates (probabilities)
    Made specifically for ion and heating ONLY.
    
    Attributes
    -----------
    E_spec : Spectrum class
        Electron energy and dNdE spectrum
    n : int
        The bin of interest in the spectrum
    den : ndarray
        density of [H, He, He+] in cm^-3
    rates : dN/dt; Filled with various rates in s^-1
    '''
    
    def __init__(self, E_spec, n, den=[1,1,1], rates=np.zeros(4)):
        '''Return a Rate object with electron of energy *E_spec.eng[n]*
        in a universe with densities *den*; initializes *rate* vector'''
        self.energy = E_spec.eng[n]
        self.n = n
        self.dNdE = E_spec.dNdE[n]
        self.den = den
        self.rates = rates
        self.v = np.sqrt(2*np.array([E_spec.eng[n]])/physics.me) 
        self.mult = self.den*self.v
    
    def ion_rate_calc(self):
        '''Fills *self.rate[1:3]* vector by calculating xsec and then converting to rate
        '''
        ion_rates = xsecs.ionize_cs(self.energy*np.ones(3),np.array([1,2,3]))*self.mult
        self.rates = np.append([0], ion_rates)
        return self.rates
    
    def heat_rate_calc(self, x_e, rs):
        '''Fills *self.rate[0]* vector with fraction going to heating
            x_e and rs...
        '''
        dE = xsecs.heating_dE(self.energy, x_e, rs, nH=physics.nH)
        print(dE-1)
        delta_dNdE = np.zeros(len(E_spec.dNdE))
        np.put(delta_dNdE, self.n, self.dNdE)
        delta = Spectrum(E_spec.eng, delta_dNdE, rs)
        shift_delta_eng = E_spec.eng+dE
        delta.shift_eng(shift_delta_eng)
        delta.rebin(E_spec.eng)
        
        heating_frac = delta.dNdE[self.n]/self.dNdE
        self.rates[0] = 1-heating_frac
        return(self.rates)
    def tot_sum(self):
        '''Sums all elements of rate
        '''
        tot_sum = sum(self.rates)
        return tot_sum
    def con_prob(self):
        '''Finds cumulative probability of each process
        '''
        con_prob = self.rates/self.tot_sum()
        return con_prob

In [4]:
eng1 = np.logspace(0.,4.,num = 500)
dNdE1 = np.logspace(0.,5.,num = 500)
rs=1000
E_spec = Spectrum(eng1,dNdE1,rs)
photon_spec = Spectrum(E_spec.eng,np.zeros(len(E_spec.eng)),rs)
TM=np.zeros([len(photon_spec.eng)+2,len(E_spec.eng)]) #possibly make pandas
Transfer_Matrix=pd.DataFrame(TM) #or np.zeros
#print(Transfer_Matrix)
print(np.array([E_spec.eng[330]]))
print(xsecs.ionize_cs(np.array([E_spec.eng[100]]),np.ones(1)))

[ 441.86599564]
[ 0.]


In [5]:
a = Rates(E_spec, 430)
print(a.energy)
print(a.ion_rate_calc())
print(a.heat_rate_calc(0.0001, 300))
print(a.tot_sum())
print(a.con_prob())
print(a.rates)

2798.29687566
[  0.00000000e+00   5.36837606e-19   6.25760697e-19   1.09194109e-19]
[  1.44947387e-11   5.36837606e-19   6.25760697e-19   1.09194109e-19]
1.44947400144e-11
[  9.99999912e-01   3.70367186e-08   4.31715709e-08   7.53336102e-09]
[  1.44947387e-11   5.36837606e-19   6.25760697e-19   1.09194109e-19]


In [12]:
def electron_low_e(E_spec, rs, ion_frac=[0,0,0], den=[1,1,1], dt=1, all_outputs=False):
    '''
    Finds transfer matrix for a given initial electron spectrum
    Outputs: photon spectrum, transfer matrix, E_loss to each process
    '''
    #Initialize photon spectrum (ask about length)
    photon_spec = Spectrum(E_spec.eng,np.zeros(len(E_spec.eng)),rs)
    
    den[0]=(physics.nH*(1-ion_frac[0]))*(rs)**3
    den[1]=(physics.nHe*(1-ion_frac[1])*(1-ion_frac[2]))*(rs)**3
    den[2]=(physics.nHe*(1-ion_frac[1])*(ion_frac[2]))*(rs)**3 #unsure about how ionfrac works for helium
    
    for n in range(len(E_spec.eng)):
        e_rates = Rates(E_spec, n, den=den)
        e_rates.ion_rate_calc()
        e_rates.heat_rate_calc(ion_frac[1], rs)
        print(e_rates.energy)
        print(e_rates.rates)
    
    
    
    

In [25]:
electron_low_e(E_spec,100)

-1.0
1.0
[ 0.  0.  0.  0.]
-1.0
1.01862899024
[ 0.  0.  0.  0.]
-1.0
1.03760501977
[ 0.  0.  0.  0.]
-1.0
1.05693455356
[ 0.  0.  0.  0.]
-1.0
1.07662417705
[ 0.  0.  0.  0.]
-1.0
1.09668059834
[ 0.  0.  0.  0.]
-1.0
1.1171106505
[ -4.44089209850062616169e-16   0.00000000000000000000e+00
   0.00000000000000000000e+00   0.00000000000000000000e+00]
-1.0
1.13792129392
[ 0.  0.  0.  0.]
-1.0
1.1591196186
[ 0.  0.  0.  0.]
-1.0
1.18071284667
[ 0.  0.  0.  0.]
-1.0
1.20270833477
[ 0.  0.  0.  0.]
-1.0
1.2251135766
[ 0.  0.  0.  0.]
-1.0
1.24793620547
[ 0.  0.  0.  0.]
-1.0
1.27118399687
[  3.33066907387546962127e-16   0.00000000000000000000e+00
   0.00000000000000000000e+00   0.00000000000000000000e+00]
-1.0
1.29486487115
[ 0.  0.  0.  0.]
-1.0
1.3189868962
[ 0.  0.  0.  0.]
-1.0
1.34355829022
[ 0.  0.  0.  0.]
-1.0
1.3685874245
[ 0.  0.  0.  0.]
-1.0
1.39408282628
[ -2.22044604925031308085e-16   0.00000000000000000000e+00
   0.00000000000000000000e+00   0.00000000000000000000e+00]
-1.0
1.42

10.618229411
[ 0.  0.  0.  0.]
-1.0
10.8160363031
[ 0.  0.  0.  0.]
-1.0
11.0175281379
[ 0.  0.  0.  0.]
-1.0
11.2227735621
[ -2.22044604925031308085e-16   0.00000000000000000000e+00
   0.00000000000000000000e+00   0.00000000000000000000e+00]
-1.0
11.4318425013
[ 0.  0.  0.  0.]
-1.0
11.6448061837
[  1.11022302462515654042e-16   0.00000000000000000000e+00
   0.00000000000000000000e+00   0.00000000000000000000e+00]
-1.0
11.8617371645
[ 0.  0.  0.  0.]
-1.0
12.0827093504
[ 0.  0.  0.  0.]
-1.0
12.3077980251
[ -2.22044604925031308085e-16   0.00000000000000000000e+00
   0.00000000000000000000e+00   0.00000000000000000000e+00]
-1.0
12.5370798744
[ 0.  0.  0.  0.]
-1.0
12.7706330131
[ 0.  0.  0.  0.]
-1.0
13.0085370109
[  1.11022302462515654042e-16   0.00000000000000000000e+00
   0.00000000000000000000e+00   0.00000000000000000000e+00]
-1.0
13.25087292
[ 0.  0.  0.  0.]
-1.0
13.4977233023
[  1.11022302462515654042e-16   0.00000000000000000000e+00
   0.00000000000000000000e+00   0.00000000000

42.3891305734
[ -2.22044604925031308085e-16   1.51360994942576496466e-19
   3.75664840418533619482e-21   0.00000000000000000000e+00]
-1.0
43.1787972733
[ -2.22044604925031308085e-16   1.53457555752554303276e-19
   3.90869999093702456567e-21   0.00000000000000000000e+00]
-1.0
43.9831746665
[  0.00000000000000000000e+00   1.55525983851930078125e-19
   4.06193793429986511879e-21   0.00000000000000000000e+00]
-1.0
44.8025367983
[ -2.22044604925031308085e-16   1.57565727410707012328e-19
   4.21630305236301302037e-21   0.00000000000000000000e+00]
-1.0
45.6371628192
[  0.00000000000000000000e+00   1.59576257425591099297e-19
   4.37173481117045684445e-21   0.00000000000000000000e+00]
-1.0
46.4873370802
[  2.22044604925031308085e-16   1.61557067863022344075e-19
   4.52817140984419259480e-21   0.00000000000000000000e+00]
-1.0
47.3533492292
[  0.00000000000000000000e+00   1.63507675764248180728e-19
   4.68554986170882711529e-21   0.00000000000000000000e+00]
-1.0
48.23549431
[  0.00000000000000000

151.481892226
[  0.00000000000000000000e+00   2.19325864570656539295e-19
   1.35933752827809077059e-20   0.00000000000000000000e+00]
-1.0
154.303846918
[  0.00000000000000000000e+00   2.19234357395893071748e-19
   1.36807216883034218626e-20   0.00000000000000000000e+00]
-1.0
157.178371777
[  0.00000000000000000000e+00   2.19118905905835518233e-19
   1.37656923129351617380e-20   0.00000000000000000000e+00]
-1.0
160.106446132
[  0.00000000000000000000e+00   2.18979907410858450078e-19
   1.38482836761171542510e-20   0.00000000000000000000e+00]
-1.0
163.089067555
[ -2.22044604925031308085e-16   2.18817760269951270994e-19
   1.39284938544205067604e-20   0.00000000000000000000e+00]
-1.0
166.127252203
[  0.00000000000000000000e+00   2.18632863626687040248e-19
   1.40063224429206873525e-20   0.00000000000000000000e+00]
-1.0
169.222035164
[  0.00000000000000000000e+00   2.18425617152820193234e-19
   1.40817705154605796606e-20   0.00000000000000000000e+00]
-1.0
172.374470806
[  0.000000000000000

531.435915805
[  0.00000000000000000000e+00   1.77348763295126630740e-19
   1.47913505823641505130e-20   0.00000000000000000000e+00]
-1.0
541.336030297
[  3.33066907387546962127e-16   1.76444078228507405397e-19
   1.47549485719317961416e-20   0.00000000000000000000e+00]
-1.0
551.420573924
[  0.00000000000000000000e+00   1.75537096148318064088e-19
   1.47175818196167771948e-20   0.00000000000000000000e+00]
-1.0
561.692982416
[  0.00000000000000000000e+00   1.74628002931396024837e-19
   1.46792743592070467586e-20   0.00000000000000000000e+00]
-1.0
572.156755506
[  0.00000000000000000000e+00   1.73716980600210196847e-19
   1.46400499671321204762e-20   0.00000000000000000000e+00]
-1.0
582.815458123
[  0.00000000000000000000e+00   1.72804207361448166563e-19
   1.45999321579814766886e-20   0.00000000000000000000e+00]
-1.0
593.672721607
[  0.00000000000000000000e+00   1.71889857645490202261e-19
   1.45589441803977244852e-20   0.00000000000000000000e+00]
-1.0
604.732244946
[  1.110223024625156

1830.31167562
[  0.00000000000000000000e+00   1.18022345522773245264e-19
   1.11574748936947125120e-20   0.00000000000000000000e+00]
-1.0
1864.40853397
[ -2.22044604925031308085e-16   1.17223258222820806560e-19
   1.10963071001451443448e-20   0.00000000000000000000e+00]
-1.0
1899.14058236
[  0.00000000000000000000e+00   1.16427877847557075134e-19
   1.10351842118179708905e-20   0.00000000000000000000e+00]
-1.0
1934.51965374
[  0.00000000000000000000e+00   1.15636226287181893505e-19
   1.09741132669097947382e-20   0.00000000000000000000e+00]
-1.0
1970.5578015
[  1.11022302462515654042e-16   1.14848324256681096545e-19
   1.09131011282147301614e-20   0.00000000000000000000e+00]
-1.0
2007.26730356
[  0.00000000000000000000e+00   1.14064191323333028131e-19
   1.08521544859825237797e-20   0.00000000000000000000e+00]
-1.0
2044.66066658
[  0.00000000000000000000e+00   1.13283845933745443860e-19
   1.07912798607584240267e-20   0.00000000000000000000e+00]
-1.0
2082.75063019
[  1.1102230246251565

5748.03015813
[  0.00000000000000000000e+00   7.58123010072954065175e-20
   7.63534960590953314909e-21   0.00000000000000000000e+00]
-1.0
5855.11015587
[  3.33066907387546962127e-16   7.52525692785259701790e-20
   7.58507710408611773468e-21   0.00000000000000000000e+00]
-1.0
5964.18494584
[  3.33066907387546962127e-16   7.46964869306289006936e-20
   7.53504820352347082636e-21   0.00000000000000000000e+00]
-1.0
6075.29168902
[ -2.22044604925031308085e-16   7.41440398187470662360e-20
   7.48526348764375811667e-21   0.00000000000000000000e+00]
-1.0
6188.46823862
[  0.00000000000000000000e+00   7.35952136181899634617e-20
   7.43572349100012793922e-21   0.00000000000000000000e+00]
-1.0
6303.75315307
[ -2.22044604925031308085e-16   7.30499938318442866725e-20
   7.38642870046314129047e-21   0.00000000000000000000e+00]
-1.0
6421.18570906
[  2.22044604925031308085e-16   7.25083657974048679040e-20
   7.33737955638466546191e-21   0.00000000000000000000e+00]
-1.0
6540.805915
[  0.00000000000000000